<a href="https://colab.research.google.com/github/AzouzSaoussen/AzouzSaoussen/blob/main/FlaskAPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **How To Use in Production**


In [14]:
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
            

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [806 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://ngrok-agent.s3.amazonaws.com buster InRelease [17.1 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [940 B]
Hit:13 http

In [3]:
!ls


aranorm.py  drive  sample_data


In [4]:
!ngrok authtoken 2ACrflX7ISTfvzCu6OgVaSmDiPo_2iByuhwjyTdnwXfsZHGtf

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install farasapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import sys
import pickle

In [8]:
experiment_folder = '/content/drive/My Drive/sentiment_analysis_project'
sys.path.append(experiment_folder)

In [9]:
vectorizer = pickle.load(open(f'{experiment_folder}/vectorizer.pkl', 'rb'))
logistic_reg = pickle.load(open(f'{experiment_folder}/logistic_reg.pkl', 'rb'))
mnb = pickle.load(open(f'{experiment_folder}/mnb.pkl', 'rb'))
svm = pickle.load(open(f'{experiment_folder}/svm.pkl', 'rb'))
rf = pickle.load(open(f'{experiment_folder}/rf.pkl', 'rb'))
mlp = pickle.load(open(f'{experiment_folder}/mlp.pkl', 'rb'))

neu_vectorizer = pickle.load(open(f'{experiment_folder}/neu_vectorizer.pkl', 'rb'))
neu_logistic_reg = pickle.load(open(f'{experiment_folder}/neu_logistic_reg.pkl', 'rb'))
neu_mnb = pickle.load(open(f'{experiment_folder}/neu_mnb.pkl', 'rb'))
neu_svm = pickle.load(open(f'{experiment_folder}/neu_svm.pkl', 'rb'))
neu_rf = pickle.load(open(f'{experiment_folder}/neu_rf.pkl', 'rb'))
neu_mlp = pickle.load(open(f'{experiment_folder}/neu_mlp.pkl', 'rb'))

In [10]:
def predict_multi_level(X, neu_mlp, neu_clf, mlp, clf):
    #return clf.predict(vectorizer.transform(X))
    neu_y_pred = neu_clf.predict(neu_mlp.transform(X))
    if len(X[neu_y_pred == 'NonNeutral']) > 0:
        y_pred = clf.predict(mlp.transform(X[neu_y_pred == 'NonNeutral'])) # classify non neutral into positive or negative
        neu_y_pred[neu_y_pred == 'NonNeutral'] = y_pred
    
    final_y_pred = neu_y_pred
    return final_y_pred

In [11]:
def predict_ByUser(X,predict,clf,neu_clf,neu_svm,svm):
      neu_y_pred = neu_clf.predict(neu_svm.transform(X))
      if len(X[neu_y_pred == 'NonNeutral']) > 0:
        if(predict==vectorizer):
           y_pred = clf.predict(vectorizer.transform(X[neu_y_pred == 'NonNeutral'])) # classify non neutral into positive or negative
           neu_y_pred[neu_y_pred == 'NonNeutral'] = y_pred
        elif (predict==logistic_reg):
           y_pred = clf.predict(logistic_reg.transform(X[neu_y_pred == 'NonNeutral'])) # classify non neutral into positive or negative
           neu_y_pred[neu_y_pred == 'NonNeutral'] = y_pred
        elif(predict==mnb):
           y_pred = clf.predict(mnb.transform(X[neu_y_pred == 'NonNeutral'])) # classify non neutral into positive or negative
           neu_y_pred[neu_y_pred == 'NonNeutral'] = y_pred
        elif(predict==svm):
           y_pred = clf.predict(svm.transform(X[neu_y_pred == 'NonNeutral'])) # classify non neutral into positive or negative
           neu_y_pred[neu_y_pred == 'NonNeutral'] = y_pred
        elif(predict==rf):
           y_pred = clf.predict(rf.transform(X[neu_y_pred == 'NonNeutral'])) # classify non neutral into positive or negative
           neu_y_pred[neu_y_pred == 'NonNeutral'] = y_pred
        else:
           y_pred = clf.predict(mlp.transform(X[neu_y_pred == 'NonNeutral'])) # classify non neutral into positive or negative
           neu_y_pred[neu_y_pred == 'NonNeutral'] = y_pred
  
      final_y_pred = neu_y_pred
      return final_y_pred
    

# **Flask App To Serve Our Models**

In [12]:
import aranorm as aranorm
import numpy as np

from flask import Flask, flash, request, redirect, url_for
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)   


@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'input_text' not in request.form:
            flash('No text found!')
            return redirect(request.url)

        text = request.form['input_text']
        # text = stemmer.stem(text)
        text = aranorm.normalize_arabic_text(text)
        if text == '':
            return 'Please, write an Arabic sentance. Symbols and non-Arabic characters will be removed from the text....'
        print(f'text: {text}')
        predcited_sentiment = predict_multi_level(np.array([text]), neu_vectorizer, neu_svm, vectorizer, mnb)
        predcited_sentiment = str(predcited_sentiment.squeeze())
        print("Predicted Sentiment:", predcited_sentiment)
        return predcited_sentiment
    
    return '''<!doctype html>
<title>تحليل المشاعر من التغريدات</title>
<script>
function myFunction()
{
    // clear the output text box from the text
    output_text_box = document.getElementById('output_text');
    output_text_box.innerHTML = '';
   
    var elements = document.getElementsByClassName("formVal");
    var formData = new FormData(); 
    
    for(var i=0; i<elements.length; i++)
    {
        formData.append(elements[i].name, elements[i].value);
    }
    var xmlHttp = new XMLHttpRequest();
        xmlHttp.onreadystatechange = function()
        {
            if(xmlHttp.readyState == 4 && xmlHttp.status == 200)
            {
                response = xmlHttp.responseText;
                output_text_box = document.getElementById('output_text');
                console.log(response);
                output_text_box.innerHTML = response;
            }
        }
        xmlHttp.open("post", "/"); 
        xmlHttp.send(formData); 
}
</script>
<h1>تحليل المشاعر من التغريدات</h1>
<p>مثال: أحب أمي و أبي</p>
<form method=post enctype=multipart/form-data>
  <textarea id="input_text"class='formVal' rows="5" cols="50" type="text" name="input_text" placeholder="التغريدة"></textarea> <br>
  <textarea id="output_text" class='formVal' rows="5" cols="50" type="text" name="output_text" placeholder="المشاعر المتوقعة"></textarea>
  <input type="submit" value="submit_now" onclick="myFunction(); return false;">
</form>

</html>
    '''
app.run()

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


100%|██████████| 241M/241M [00:15<00:00, 15.8MiB/s]


[2022-06-08 04:22:18,872 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://602b-130-211-248-140.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Jun/2022 04:45:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2022 04:45:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2022 04:52:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2022 04:52:11] "GET /favicon.ico HTTP/1.1" 404 -


## Flask App V1.2

In [13]:
"""import aranorm as aranorm
import numpy as np

from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)   


@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'input_text' not in request.form:
            flash('No text found!')
            return redirect(request.url)
        predict = request.form.get('comp_select')
        neu_predict = request.form.get('comp_select_neutral')
        text = request.form['input_text']
        # text = stemmer.stem(text)
        text = aranorm.normalize_arabic_text(text)
        if text == '':
            return 'Please, write an Arabic sentance. Symbols and non-Arabic characters will be removed from the text....'
        print(f'text: {text}')
        predcited_sentiment = predict_multi_level(np.array([text]), neu_mlp, neu_predict, mlp, predict)
        predcited_sentiment = str(predcited_sentiment.squeeze())
    
        print("Predicted Sentiment:", predcited_sentiment)
        return predcited_sentiment
    
    return render_template('content/drive/My Drive/sentiment_analysis_project/index.html')

app.run()"""

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://35b2-130-211-248-140.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
